<h1>Tóm tắt văn bản - NLP</h1>

<h2>Extractive Summarization</h2>

In [42]:
import nltk
from underthesea import word_tokenize, sent_tokenize
import re
import numpy as np 

with open('New folder\output_2024-03-21_10-21-55.txt', 'r', encoding='utf-8') as file:
    text = file.read()


with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

# Hàm xử lý văn bản
def text_handle(text):
    lower_text = text.lower()
    clean_text = re.sub(r'\s+', ' ', lower_text)
    return clean_text.strip()

# Hàm loại bỏ từ dừng
def remove_stopwords(sentence, stopwords):
    words = word_tokenize(sentence)
    filtered_sentence = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_sentence)

# Hàm loại bỏ dấu câu 
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]','',sentence)

# Tách các câu trong văn bản
sentences = sent_tokenize(text_handle(text))

# Xử lý từng câu
processed_sentences = []
for sentence in sentences: 
    cleaned_sentence = remove_stopwords(sentence, stopwords)
    cleaned_sentence = remove_punctuation(cleaned_sentence)
    processed_sentences.append(cleaned_sentence)


print("Các câu sau khi xử lý:")
print(processed_sentences)


Các câu sau khi xử lý:
['vinif kỳ vọng trẻ  coi khoa học nghề lương   vnexpress  2132024 khu vực hà nộitp hồ chí minh international thời sự góc thế giới video podcasts kinh doanh bất động sản khoa học giải trí thể thao pháp luật giáo dục sức khỏe đời sống du lịch hóa xe ý kiến tâm sự thư giãn trở lại khoa học khoa học  2132024  09  00  gmt  7  vinif kỳ vọng trẻ  coi khoa học nghề lương  quỹ đổi mới sáng tạo vingroup  vinif  tài trợ 220 tỷ đồng 5  nguồn lực nghiên cứu khoa học nghề nghiệp hấp dẫn trẻ ', 'kinh phí tài trợ 220 tỷ đồng học viên  nghiên cứu sinh tiến sĩ trẻ ', ' quỹ góp phần 500 báo công bố tạp chí uy tín quốc tế  1000 báo chuyên ngành báo cáo hội thảo  hàng chục sáng chế  giải pháp hữu ích  hàng trăm giải thưởng khoa học công nghệ ', 'đi nhân lực khoa học công nghệ trẻ trung  chất lượng 1500 thành viên gia tăng  gs ', 'vũ hà văn  giám đốc khoa học quỹ vinif    coi nghiên cứu khoa học nghề   vốn hiển nhiên nhà khoa học  nhà nghiên cứu trẻ phát triển thế giới ', 'học viên  n

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Hàm tính điểm TF-IDF cho từng câu
def calculate_tfidf(sentences):
    tfidf_vector = TfidfVectorizer()
    tfidf_matrix = tfidf_vector.fit_transform(sentences)
    tfidf_score = np.asarray(tfidf_matrix.sum(axis=1)).ravel()
    return tfidf_score

# Hàm tính điểm cho mỗi câu dựa trên chiều dài và điểm TF-IDF
def calculate_sentence_scores(sentences):
    tfidf = calculate_tfidf(sentences)
    sentence_lengths = [len(sentence.split()) for sentence in sentences]
    max_sentence_length = max(sentence_lengths)
    # Chuẩn hóa độ dài mỗi câu
    normalized_lengths = [length / max_sentence_length for length in sentence_lengths]
    # Tính điểm cho mỗi câu bằng cách kết hợp điểm TF-IDF và độ dài chuẩn hóa của câu
    sentence_scores = [tfidf_score * length for tfidf_score, length in zip(tfidf, normalized_lengths)]
    return sentence_scores

sentence_scores = calculate_sentence_scores(processed_sentences)

# Hiển thị điểm của từng câu
for i, sentence in enumerate(processed_sentences):
    print("Câu {}: {}".format(i+1, sentence))
    print("Điểm: {:.2f}".format(sentence_scores[i]))


Câu 1: vinif kỳ vọng trẻ  coi khoa học nghề lương   vnexpress  2132024 khu vực hà nộitp hồ chí minh international thời sự góc thế giới video podcasts kinh doanh bất động sản khoa học giải trí thể thao pháp luật giáo dục sức khỏe đời sống du lịch hóa xe ý kiến tâm sự thư giãn trở lại khoa học khoa học  2132024  09  00  gmt  7  vinif kỳ vọng trẻ  coi khoa học nghề lương  quỹ đổi mới sáng tạo vingroup  vinif  tài trợ 220 tỷ đồng 5  nguồn lực nghiên cứu khoa học nghề nghiệp hấp dẫn trẻ 
Điểm: 8.07
Câu 2: kinh phí tài trợ 220 tỷ đồng học viên  nghiên cứu sinh tiến sĩ trẻ 
Điểm: 0.57
Câu 3:  quỹ góp phần 500 báo công bố tạp chí uy tín quốc tế  1000 báo chuyên ngành báo cáo hội thảo  hàng chục sáng chế  giải pháp hữu ích  hàng trăm giải thưởng khoa học công nghệ 
Điểm: 1.96
Câu 4: đi nhân lực khoa học công nghệ trẻ trung  chất lượng 1500 thành viên gia tăng  gs 
Điểm: 0.68
Câu 5: vũ hà văn  giám đốc khoa học quỹ vinif    coi nghiên cứu khoa học nghề   vốn hiển nhiên nhà khoa học  nhà nghiên c

In [51]:
def summarize_text(sentences, sentence_scores, num_sentences):
    # Sắp xếp các câu theo điểm số giảm dần
    sorted_indices = sorted(range(len(sentence_scores)), key=lambda i: sentence_scores[i], reverse=True)
    # Chọn ra các câu có điểm cao nhất để tạo tóm tắt
    summary_indices = sorted_indices[:num_sentences]
    # Sắp xếp lại các chỉ số câu theo thứ tự xuất hiện trong văn bản
    summary_indices.sort()
    # Tạo tóm tắt bằng cách lấy các câu được chọn
    summary = [sentences[i] for i in summary_indices]
    return summary


num_summary_sentences = 6
summary = summarize_text(processed_sentences, sentence_scores, num_summary_sentences)


print("Tóm tắt văn bản:")
for sentence in summary:
    print(sentence)


Tóm tắt văn bản:
vinif kỳ vọng trẻ  coi khoa học nghề lương   vnexpress  2132024 khu vực hà nộitp hồ chí minh international thời sự góc thế giới video podcasts kinh doanh bất động sản khoa học giải trí thể thao pháp luật giáo dục sức khỏe đời sống du lịch hóa xe ý kiến tâm sự thư giãn trở lại khoa học khoa học  2132024  09  00  gmt  7  vinif kỳ vọng trẻ  coi khoa học nghề lương  quỹ đổi mới sáng tạo vingroup  vinif  tài trợ 220 tỷ đồng 5  nguồn lực nghiên cứu khoa học nghề nghiệp hấp dẫn trẻ 
 đầu  báo công bố tạp chí quốc tế uy tín bước đầu phát triển sản phẩm ứng dụng nghiên cứu  công cụ bàn chân đái tháo đường việt nam   bão  giám đốc khoa học quỹ vinif phân tích  nghề  tất nhiên gắn quyền lợi trách nhiệm 
chương trình  hội thảo khoa học góc  giảng đại chúng phát triển bền vững  xác định hướng đi phát triển tác động công nghệ  toán học quốc tế 2024  loạt sự kiện thu hút hàng chục nghìn tham dự  hàng trăm nghìn kênh online trường  viện  báo chí viết  tiếp sóng  ts 
học bổng tiến sĩ q